#  import packges

In [10]:
import configparser
import psycopg2
import pandas as pd
import boto3
import json
import time
from botocore.exceptions import ClientError

## Enter ACCESS KEY details in the config file dwh.cfg before executing the cell below

#KEY= <YOUR_AWS_KEY>
#SECRET= <YOUR_AWS_SECRET>

# Load DWH Params from a file

In [11]:
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    KEY                    = config.get("ACCESS","KEY")
    SECRET                 = config.get("ACCESS","SECRET")

    DWH_CLUSTER_TYPE       = 'multi-node'
    DWH_NUM_NODES          = 8
    DWH_NODE_TYPE          = 'dc2.large'
    
    DWH_CLUSTER_IDENTIFIER = 'udacity-project-cluster'
    DWH_DB                 = config.get("CLUSTER","DB_NAME")
    DWH_DB_USER            = config.get("CLUSTER","DB_USER")
    DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
    DWH_PORT               = config.get("CLUSTER","DB_PORT")

    DWH_IAM_ROLE_NAME      = config.get("IAM_ROLE", "ROLE_NAME")

    (DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

    pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,udacity-project-cluster
4,DWH_DB,sparkify
5,DWH_DB_USER,student
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


# Create clients for IAM, EC2, S3 and Redshift
**Note**: We are creating these resources in the the **us-west-2** region. Choose the same region in the your AWS web console to the see these resources.

In [12]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# DELETE  Redshift Cluster that was created 

In [13]:
    try:
    #Uncomment & run to delete the created resources
        (redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
           )
    except Exception as e:
        print(e)

##  *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Deleted` or 'CusterNotFound' error

In [19]:
#cluster status 
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster udacity-project-cluster not found.

# Detach S3 read policy that was attached to the IAM role
# Delete the IAM role

In [17]:
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")

{'ResponseMetadata': {'RequestId': '3aa49ee1-06ec-4ccd-b5ee-a8e04417cdcf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3aa49ee1-06ec-4ccd-b5ee-a8e04417cdcf',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Sat, 25 Mar 2023 11:50:29 GMT'},
  'RetryAttempts': 0}}

In [18]:
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)

{'ResponseMetadata': {'RequestId': 'd8176255-3d5f-41cd-a86a-e856a48bc61c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd8176255-3d5f-41cd-a86a-e856a48bc61c',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Sat, 25 Mar 2023 11:50:34 GMT'},
  'RetryAttempts': 0}}